In [88]:
import numpy as np
import scipy as sc
from scipy import sparse
from scipy.special import expit


class BaseSmoothOracle:
    """
    Базовый класс для реализации оракулов.
    """
    def func(self, w):
        """
        Вычислить значение функции в точке w.
        """
        raise NotImplementedError('Func oracle is not implemented.')

    def grad(self, w):
        """
        Вычислить значение градиента функции в точке w.
        """
        raise NotImplementedError('Grad oracle is not implemented.')


class BinaryLogistic(BaseSmoothOracle):
    """
    Оракул для задачи двухклассовой логистической регрессии.

    Оракул должен поддерживать l2 регуляризацию.
    """

    def __init__(self, l2_coef):
        """
        Задание параметров оракула.

        l2_coef - коэффициент l2 регуляризации
        """
        self.l2_coef = l2_coef
        pass

    def func(self, X, y, w):
        """
        Вычислить значение функционала в точке w на выборке X с ответами y.

        X - scipy.sparse.csr_matrix или двумерный numpy.array

        y - одномерный numpy array

        w - одномерный numpy array
        """
        if sc.sparse.issparse(X):
            sp_mtr = sparse.csr_matrix(-y).multiply(sparse.csr_matrix(w).dot(X.transpose()))
            return 1 / y.size *\
                (np.sum(np.log(np.exp(sp_mtr.data) + 1)) + (sp_mtr.shape[1] - sp_mtr.data.shape[0]) * np.log(2))+\
                self.l2_coef / 2 * np.linalg.norm(w) ** 2
        else:
            return 1 / y.size *\
                np.sum(np.log(1 + np.exp(-y.reshape(1, -1) *
                                         np.dot(w, X.T)))) +\
                self.l2_coef / 2 * np.linalg.norm(w) ** 2
        return super().func(w)

    def grad(self, X, y, w):
        """
        Вычислить градиент функционала в точке w на выборке X с ответами y.

        X - scipy.sparse.csr_matrix или двумерный numpy.array

        y - одномерный numpy array

        w - одномерный numpy array
        """
        if sc.sparse.issparse(X):
            margin = sparse.csr_matrix(y).multiply(sparse.csr_matrix(w).dot(X.transpose()))
            niz = margin.copy()
            niz.data = expit(margin.data)
            margin.data = np.exp(-margin.data)
            if np.count_nonzero(w):
                return - 1 / y.size * \
                    np.array((margin.multiply(sparse.csr_matrix(y)).multiply(X.transpose()).multiply(niz)).sum(axis=1)).ravel() +\
                    self.l2_coef * w
            else:
                return - 1 / y.size * \
                    (np.array((margin.multiply(sparse.csr_matrix(y)).multiply(X.transpose()).multiply(niz)).sum(axis=1)).ravel() +
                    0.5 * np.array(sparse.csr_matrix(y).multiply(X.transpose()).sum(axis=1)).ravel()) +\
                    self.l2_coef * w
        else:
            return - 1 / y.size * \
                np.sum(np.exp(-y * np.dot(w, X.T)).reshape(-1, 1) *
                       y.reshape(-1, 1) * X /
                       np.array(1 + np.exp(-y * np.dot(w, X.T))).reshape(-1, 1),
                       axis=0) + self.l2_coef * w
        return super().grad(w)


In [91]:
X = np.array([[0, 1, 2, 0, 0], [1, 0, 0, 3, 0], [0, 0, 0, 0, 0]])
y = np.array([1, 1, -1])
w_0 = np.zeros(5)
clf = BinaryLogistic(l2_coef=0.1)
w = w_0.copy()
w_k = w_0.copy()
for k in range(1, max_iter):
    print(clf.grad(X, y, w_k))
    w_k1 = w_k - alpha / (k ** beta) * clf.grad(X, y, w_k)
    print(clf.func(X, y, w_k1))    
    w_k = w_k1.copy()
# print('\n', clf.grad(X, y, -np.array([-0.16666667, -0.16666667, -0.33333333, -0.5, 0.])))

[-0.16666667 -0.16666667 -0.33333333 -0.5         0.        ]
0.42984666179332925
[-0.0362897  -0.08431357 -0.16862714 -0.1088691   0.        ]
0.392155519111847
[-0.0184156  -0.04885293 -0.09770587 -0.05524679  0.        ]
0.37978971531719696
[-0.01070454 -0.03086667 -0.06173334 -0.03211362  0.        ]
0.37491359404310054


In [92]:
X = np.array([[0, 1, 2, 0, 0], [1, 0, 0, 3, 0], [0, 0, 0, 0, 0]])
X = sparse.csr_matrix(X)
y = np.array([1, 1, -1])
w_0 = np.zeros(5)
clf = BinaryLogistic(l2_coef=0.1)
w = w_0.copy()
w_k = w_0.copy()
for k in range(1, max_iter):
    print(clf.grad(X, y, w_k))
    w_k1 = w_k - alpha / (k ** beta) * clf.grad(X, y, w_k)
    print(clf.func(X, y, w_k1))    
    w_k = w_k1.copy()
# print('\n', clf.grad(X, y, -np.array([-0.16666667, -0.16666667, -0.33333333, -0.5, 0.])))

[-0.16666667 -0.16666667 -0.33333333 -0.5         0.        ]
0.42984666179332925
[-0.0362897  -0.08431357 -0.16862714 -0.1088691   0.        ]
0.392155519111847
[-0.0184156  -0.04885293 -0.09770587 -0.05524679  0.        ]
0.37978971531719696
[-0.01070454 -0.03086667 -0.06173334 -0.03211362  0.        ]
0.37491359404310054


In [86]:
np.random.seed(10)
alpha=1
beta=0
tolerance=1e-4
max_iter=5
l2_coef=0.1
l, d = 1000, 10
X = np.random.random((l, d))
y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)
w_0 = np.zeros(d)
# history = clf.fit(X, y, w_0=np.zeros(d), trace=True)
# print(' '.join([str(x) for x in history['func']]))
# clf.get_weights()
clf = BinaryLogistic(l2_coef=0.1)
w = w_0.copy()
w_k = w_0.copy()
for k in range(1, max_iter):
    print(clf.grad(X, y, w_k))
    w_k1 = w_k - alpha / (k ** beta) * clf.grad(X, y, w_k)

    print(clf.func(X, y, w_k1))
    
#     if np.absolute(clf.func(X, y, w_k1) -
#                    clf.func(X, y, w_k)) < tolerance:
#         break
    w_k = w_k1.copy()


[ 0.00027869  0.00711611  0.01160488 -0.00086325  0.00926855  0.00675723
  0.00278599  0.00450789  0.01230398  0.01148879]
0.6926868443805101
[-0.00363342  0.00210854  0.00617056 -0.00475758  0.00407212  0.00201085
 -0.00138171  0.00013195  0.00679131  0.0061085 ]
0.6925235796993467
[-0.00425329  0.00071929  0.00431978 -0.00527168  0.00245644  0.00068436
 -0.0022579  -0.00093611  0.00486688  0.00426456]
0.6924122120106758
[-0.00402582  0.00032368  0.00349503 -0.00492685  0.00184626  0.00029926
 -0.00226477 -0.00111297  0.00397612  0.00343609]
0.6923275050752085


In [87]:
np.random.seed(10)
alpha=1
beta=0
tolerance=1e-4
max_iter=5
l2_coef=0.1
l, d = 1000, 10
X = np.random.random((l, d))
X = sparse.csr_matrix(X)
y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)
w_0 = np.zeros(d)
# history = clf.fit(X, y, w_0=np.zeros(d), trace=True)
# print(' '.join([str(x) for x in history['func']]))
# clf.get_weights()
clf = BinaryLogistic(l2_coef=0.1)
w = w_0.copy()
w_k = w_0.copy()
for k in range(1, max_iter):
    print(clf.grad(X, y, w_k))
    w_k1 = w_k - alpha / (k ** beta) * clf.grad(X, y, w_k)

    print(clf.func(X, y, w_k1))
    
#     if np.absolute(clf.func(X, y, w_k1) -
#                    clf.func(X, y, w_k)) < tolerance:
#         break
    w_k = w_k1.copy()


[ 0.00027869  0.00711611  0.01160488 -0.00086325  0.00926855  0.00675723
  0.00278599  0.00450789  0.01230398  0.01148879]
0.6926868443805101
[-0.00363342  0.00210854  0.00617056 -0.00475758  0.00407212  0.00201085
 -0.00138171  0.00013195  0.00679131  0.0061085 ]
0.6925235796993467
[-0.00425329  0.00071929  0.00431978 -0.00527168  0.00245644  0.00068436
 -0.0022579  -0.00093611  0.00486688  0.00426456]
0.6924122120106759
[-0.00402582  0.00032368  0.00349503 -0.00492685  0.00184626  0.00029926
 -0.00226477 -0.00111297  0.00397612  0.00343609]
0.6923275050752085


In [97]:
if sc.sparse.issparse(X):
    print(type(np.exp(sparse.csr_matrix(-y).multiply(sparse.csr_matrix(w).dot(X.transpose())).todense())))
else:
    print((np.exp(-y.reshape(1, -1) * np.dot(w, X.T))))

<class 'numpy.matrix'>


In [2]:
bl = BinaryLogistic(1)

In [3]:
X = np.array([[3, 3, 3], [0, 0, 0], [1, 1, 1]])
y = np.array([1, -1, -1])
w = np.array([5, 3, 2])

In [4]:
%%timeit
np.random.seed(4181)
l2_coef = np.random.randint(0, 10)
l, d = 1000, 10
my_oracle = BinaryLogistic(l2_coef=l2_coef)
X = sc.sparse.csr_matrix(np.random.random((l, d)))
# X = np.random.random((l, d))

y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)
res = my_oracle.func(X, y, w)
# print(res)

7.85 ms ± 2.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
len(res)

NameError: name 'res' is not defined

In [6]:
l2_coef

NameError: name 'l2_coef' is not defined

In [7]:
w.shape

(3,)

In [8]:
my_oracle = BinaryLogistic(l2_coef=1)
print(my_oracle.func(np.zeros((3, 3)), np.ones(3), np.zeros(3)))


0.6931471805599452


In [9]:
my_oracle.func(X, y, w)

22.564397526486417

In [10]:
my_oracle.grad(X, y, w)

array([5.3333182, 3.3333182, 2.3333182])

In [145]:
np.random.seed(4181)
l2_coef = np.random.randint(0, 10)
l, d = 1000, 10
my_oracle = BinaryLogistic(l2_coef=l2_coef)
X = sc.sparse.csr_matrix(np.random.random((l, d)))
# X = np.random.random((l, d))

y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)
# res = my_oracle.grad(X, y, w)
# print(res)

In [133]:
# -y * np.dot(w, np.array(X.todense().T))
ress = (np.exp(-y * np.dot(w, np.array(X.todense().T))).reshape(-1, 1) * y.reshape(-1, 1) * np.array(X.todense()) /\
  np.array(1 + np.exp(-y * np.dot(w, np.array(X.todense().T)))).reshape(-1, 1))
ress.sum(axis=0)
# expit(-y * np.dot(w, np.array(X.todense().T)))

array([-194.59767962, -206.42852773, -208.49810813, -197.20682332,
       -200.16780443, -208.74661492, -202.94829964, -196.89933998,
       -201.4572312 , -214.9538811 ])

In [155]:
margin = sparse.csr_matrix(y).multiply(sparse.csr_matrix(w).dot(X.transpose()))

niz = margin.copy()

niz.data = expit(margin.data)

margin.data = np.exp(-margin.data)

verh = margin.multiply(sparse.csr_matrix(y)).multiply(X.transpose()) 

res = np.array(verh.multiply(niz).todense())

ok_res = np.array(margin.multiply(sparse.csr_matrix(y)).multiply(X.transpose()).multiply(niz).todense()).sum(axis=1)

# # np.sum()
# np.sum(np.log(np.exp(sparse.csr_matrix(-y).multiply(sparse.csr_matrix(w).dot(X.transpose())).data) + 1))
# niz.todense()
ok_res.sum()

-2031.9043100789922

In [156]:
np.array(margin.multiply(sparse.csr_matrix(y)).multiply(X.transpose()).multiply(niz).todense()).sum(axis=1)

array([-194.59767962, -206.42852773, -208.49810813, -197.20682332,
       -200.16780443, -208.74661492, -202.94829964, -196.89933998,
       -201.4572312 , -214.9538811 ])

In [181]:
I = sparse.csr_matrix(np.ones((X.shape[0],1)))
np.array((margin.multiply(sparse.csr_matrix(y)).multiply(X.transpose()).multiply(niz)*I).todense()).ravel()

array([-194.59767962, -206.42852773, -208.49810813, -197.20682332,
       -200.16780443, -208.74661492, -202.94829964, -196.89933998,
       -201.4572312 , -214.9538811 ])

In [ ]:
if sc.sparse.issparse(X):
            margin = sparse.csr_matrix(y).multiply(sparse.csr_matrix(w).dot(X.transpose()))
            niz = margin.copy()
            niz.data = expit(margin.data)
            margin.data = np.exp(-margin.data)
            I = sparse.csr_matrix(np.ones((X.shape[0],1)))
            return - 1 / y.size * \
                np.array((margin.multiply(sparse.csr_matrix(y)).multiply(X.transpose()).multiply(niz)*I).todense()).ravel() +\
                self.l2_coef * w

In [13]:
# np.sum(np.exp(-y * np.dot(w, np.array(X.todense().T))).reshape(-1, 1) *
#                        y.reshape(-1, 1) * np.array(X.todense()) /
#                        np.array(1 + np.exp(-y * np.dot(w, np.array(X.todense().T)))).reshape(-1, 1),
#                        axis=0)

# np.sum(np.exp())

from scipy.special import expit, logsumexp
expit

a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[0, 2, 0], [3, 0, 0]])
c = np.array([5, 10, 100])
print(a*b)
ys = sparse.csr_matrix(a)
xz = sparse.csr_matrix(b)
xc = sparse.csr_matrix(c)
# y * w.dot(X.transpose())
# print(ys, '\n', xz)
# type(ys.data * xz.data)
# xz.dot(ys)
print(np.sum(b/c, axis=1))
# print(b.dot(c))
# print(xz.dot(xc.transpose()))
# xz.shape


[[ 0  4  0]
 [12  0  0]]
[0.2 0.6]


In [14]:
%%timeit
a.dot(c)

3.08 µs ± 580 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [146]:
%%timeit
my_oracle.func(X, y, w)

2.65 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [148]:
%%timeit
my_oracle.grad(X, y, w)

6.49 ms ± 446 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [149]:
np.random.seed(4181)
l2_coef = np.random.randint(0, 10)
l, d = 1000, 10
my_oracle = BinaryLogistic(l2_coef=l2_coef)
# X = sc.sparse.csr_matrix(np.random.random((l, d)))
X = np.random.random((l, d))

y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)

In [150]:
%%timeit
my_oracle.func(X, y, w)

210 µs ± 26.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [151]:
%%timeit
my_oracle.grad(X, y, w)

541 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
